In [1]:
import pylab
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
flow = pd.read_csv("flow.tsv", na_values=['-'], delimiter="\t", error_bad_lines=False)
occupancy = pd.read_csv("occupancy.tsv", na_values=['-'], delimiter="\t", error_bad_lines=False)
speed = pd.read_csv("speed.tsv", na_values=['-'], delimiter="\t", error_bad_lines=False)
timestamp = pd.read_csv("timestamp.tsv", na_values=['-'], delimiter="\t", error_bad_lines=False)

In [ ]:
flow.head()

In [3]:
df = pd.concat([flow,occupancy,speed,timestamp], axis=1)

In [ ]:
# df.head()

In [4]:
df.columns = ['flow1','occupancy1','speed1','timestamp']
df.head()

flow1  occupancy1  speed1            timestamp
0      9         1.0    51.0  2009-08-19T15:06:41
1     10         2.0    48.6  2009-08-19T15:07:40
2     12         1.0    48.6  2009-08-19T15:08:41
3      9         2.0    48.6  2009-08-19T15:09:41
4      6         1.0    48.6  2009-08-19T15:10:41

In [7]:
df1 =  pd.DataFrame({"flow" : df.flow1, "occupancy" : df.occupancy1,"speed" : df.speed1,"timestamp" : df.timestamp,"detector" : "x"})
# df2 = pd.DataFrame({"flow" : df.flow2, "occupancy" : df.occupancy2,"speed" : df.speed2,"timestamp" : df.timestamp,"detector" : "y"})
# df3 = pd.DataFrame({"flow" : df.flow3, "occupancy" : df.occupancy3,"speed" : df.speed3,"timestamp" : df.timestamp,"detector" : "z"})
# df3 = pd.DataFrame({"flow" : df.flow4, "occupancy" : df.occupancy4,"speed" : df.speed4,"timestamp" : df.timestamp,"detector" : "x1"})

In [8]:
# frames = [df1]
result = df1

In [9]:
result = result.sort_values("timestamp")

In [10]:
result.head(10)

detector  flow  occupancy  speed            timestamp
0        x     9        1.0   51.0  2009-08-19T15:06:41
1        x    10        2.0   48.6  2009-08-19T15:07:40
2        x    12        1.0   48.6  2009-08-19T15:08:41
3        x     9        2.0   48.6  2009-08-19T15:09:41
4        x     6        1.0   48.6  2009-08-19T15:10:41
5        x    12        2.0   47.4  2009-08-19T15:11:40
6        x     9        1.0   47.4  2009-08-19T15:12:40
7        x    10        1.0   48.6  2009-08-19T15:13:40
8        x    13        3.0   45.6  2009-08-19T15:14:40
9        x    10        2.0   46.8  2009-08-19T15:15:41

In [11]:
# Finding outlier values for Occoupancy
val = np.sort(result["occupancy"].unique())
plt.plot(val)

# so keeping occupancy threshold as 120 since all values above them show a steep change indicating possible outliers.
# jam_occupancy_threshold = 120

In [14]:
print np.sort(result["flow"].unique())

[-58 -49 -47 -45 -44 -42 -41 -40 -39 -38 -37 -36 -35 -34 -33 -32 -31 -30
 -29 -28 -27 -26 -25 -24 -23 -22 -21 -20 -19 -18 -17 -16 -15 -14 -13 -12
 -11 -10  -9  -8  -7  -6  -5  -4  -3  -2  -1   0   1   2   3   4   5   6
   7   8   9  10  11  12  13  14  15  16  17  18  19  20  21  22  23  24
  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42
  43  44  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60
  63  69]


In [15]:
# Outlier speed visualization
plt.figure(figsize=(10,10))
# plt.xlim(550, 650)
plt.plot(np.sort(result["occupancy"].unique()))

In [ ]:
# result[(pd.isnull(result['speed'])) | (pd.isnull(result['occupancy'])) | (pd.isnull(result['flow']))]

In [16]:
removeResult1 = result[(result['speed'] < 0) | (result['flow'] < 0)]
removeResult2 = result[(result['flow'] == 0) & (result['speed'] > 0)]
jam_occupancy_threshold = 100
removeResult3 = result[((result['speed'] == 0) & 
                            (result['flow'] == 0) & 
                            (result['occupancy'] != 0) & 
                            (result['occupancy'] < jam_occupancy_threshold))]
# skewed data
removeResult4 = result[(result['speed'] == 0) & (result['flow'] == 0) & (result["occupancy"] == 0)]
removeResult5 = result[(result['speed'] == 0) & (result['flow'] != 0) & (result["occupancy"] == 0)]
removeResult6 = result[(pd.isnull(result['speed'])) | (pd.isnull(result['occupancy'])) | (pd.isnull(result['flow']))]
# removeResult7 = result[result['occupancy'] < jam_occupancy_threshold] # occupancy increases rapidly after 80, mostly outliers

In [17]:
result.head()

detector  flow  occupancy  speed            timestamp
0        x     9        1.0   51.0  2009-08-19T15:06:41
1        x    10        2.0   48.6  2009-08-19T15:07:40
2        x    12        1.0   48.6  2009-08-19T15:08:41
3        x     9        2.0   48.6  2009-08-19T15:09:41
4        x     6        1.0   48.6  2009-08-19T15:10:41

In [18]:
result = pd.concat([result,removeResult1,removeResult1]).drop_duplicates(keep=False)
result = pd.concat([result,removeResult2,removeResult2]).drop_duplicates(keep=False)
result = pd.concat([result,removeResult3,removeResult3]).drop_duplicates(keep=False)
result = pd.concat([result,removeResult4,removeResult4]).drop_duplicates(keep=False)
result = pd.concat([result,removeResult5,removeResult5]).drop_duplicates(keep=False)
result = pd.concat([result,removeResult6,removeResult6]).drop_duplicates(keep=False)
# result = pd.concat([result,removeResult7,removeResult7]).drop_duplicates(keep=False)

In [19]:
result = result.sort_values(["flow"])

In [20]:
def sampleData(data, threshold, columnName):
    flowBins = np.zeros(shape=(20,), dtype = object)
    data = data.sort_values(columnName)
    difference = threshold
    high  = difference
    low = 0;
    for i in range(0, 20, 1):
        b = data[data[columnName] < high]
        b = b[b[columnName] >= low]
        flowBins[i] = b
        low = high
        high = high + difference
    return flowBins

In [21]:
flowBins = np.zeros(shape=(20,), dtype = object)
flowBins = sampleData(result, 4 ,"flow")    

In [22]:

flow_occupancy_bin = np.ndarray(shape=(20,20), dtype = object)
for i in range(0,20,1):
    data = pd.DataFrame(flowBins[i])
    flow_occupancy_bin[i] = sampleData(data, 10 ,"occupancy") 

In [23]:
flow_occupancy_speed_bin = np.ndarray(shape=(20,20,20), dtype = object)
for i in range(0,20,1):
    for j in range(0,20,1):
        data = flow_occupancy_bin[i][j];
        flow_occupancy_speed_bin[i][j] = sampleData(data, 11 ,"speed") 

In [24]:
!mkdir output

In [25]:
final_result = pd.DataFrame(columns = ["detector","flow","occupancy","speed","timestamp", "probability"])
total_points = len(result)
#final_result = ["0" , "0", "0", "0"]
for i in range(0,20,1):
    for j in range(0,20,1):
        for k in range(0,20,1):
            prob_data = flow_occupancy_speed_bin[i][j][k]
            points_inside_box = prob_data.shape[0]
            f = prob_data.flow.max() - prob_data.flow.min()
            o = prob_data.occupancy.max() - prob_data.occupancy.min()
            s = prob_data.speed.max() - prob_data.speed.min()
            #volume = f*o*s
            volume = 440
            if(volume == 0): volume =1
            probability = points_inside_box/float(total_points * volume)
#             print(prob_data)
            prob_data["probability"] = probability
            prob_data.to_csv("output/out" + str(i) + '-' + str(j) + '-' + str(k) + ".csv",sep="\t")
#             print(prob_data.head(1))
#           final_result.add([prob_data['flow'], prob_data['speed'], prob_data['occupancy'], prob_data['probability']])
            #final_result_align, prob_data_align = final_result.align(prob_data,axis=1)
            #final_result = pd.concat([final_result_align, prob_data_align])
            #final_result = np.append(final_result, [prob_data['flow'], prob_data['speed'], prob_data['occupancy'], prob_data['probability']])

In [26]:
! cat output/*.csv > merged.csv

In [27]:
!rm -rf output/

In [28]:
merged = pd.read_csv("merged.csv", na_values=['-'], delimiter="\t", error_bad_lines=False)

/Users/abhisek/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (2,3,4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [29]:
removeHeader = merged[merged["probability"] == "probability"]

In [30]:
merged = pd.concat([merged,removeHeader,removeHeader]).drop_duplicates(keep=False)

In [31]:
merged.tail()

Unnamed: 0 detector flow occupancy speed            timestamp  \
1477467    126092.0        x   36      10.0  50.4  2009-11-17T06:29:04   
1477675    569208.0        x   39      21.0  18.0  2010-11-04T09:04:44   
1477676    631186.0        x   38      24.0  21.6  2011-02-22T09:43:31   
1477688    146242.0        x   38      28.0  22.2  2009-12-01T08:06:19   
1477689    106261.0        x   36      24.0  27.0  2009-11-03T09:00:53   

               probability  
1477467  3.09248675395e-09  
1477675  3.09248675395e-09  
1477676  3.09248675395e-09  
1477688  3.09248675395e-09  
1477689  3.09248675395e-09

In [32]:
# merged = merged.reset_index(drop=True)
merged["probability"] = merged["probability"].apply(pd.to_numeric)
merged = merged.sort_values(["probability"])
final_columns = ['flow', 'speed', 'occupancy', 'probability']
merged = merged[final_columns]

In [33]:
merged.tail()

flow speed occupancy  probability
475653    4  52.8         1     0.000498
475654    6  52.8         1     0.000498
475655    5  52.8         1     0.000498
475631    4  52.8         1     0.000498
560855    4  54.0       1.0     0.000498

In [34]:
# merged
final_result = pd.DataFrame(columns = ["flow","speed","occupancy", "probability"])
for i in range(0,len(merged),100):
    final_result = final_result.append(merged.iloc[[i]])

In [35]:
final_result.to_csv("3451.txt",sep="\t", index=False)

In [36]:
final_result.ta()

AttributeError: 'DataFrame' object has no attribute 'ta'

In [ ]:
# len(merged)